# Get start_Step1

Define a function for solving the NCPO problems with given standard deviations of process noise and observtion noise, length of  estimation data and required relaxation level.

* mount drive
* set envirment
* install packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Causality_NotesTest/")

Mounted at /content/drive


In [ ]:
!export PYTHONPATH="$PYTHONPATH:/content/drive/MyDrive/Colab Notebooks/Causality_NotesTest"
import os
os.environ['MOSEKLM_LICENSE_FILE']="/content/drive/MyDrive/Colab Notebooks"

In [ ]:
### for colab ###
# To execute the notebook directly in colab make sure your MOSEK license file is in one the locations
#
# /content/mosek.lic   or   /root/mosek/mosek.lic
#
# inside this notebook's internal filesystem.
# Install MOSEK and ncpol2sdpa if not already installed
!pip install mosek torch
!pip install ncpol2sdpa
!pip install networkx
!pip install gcastle==1.0.3rc2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ncpol2sdpa: filename=ncpol2sdpa-1.12.2-py3-none-any.whl size=70796 sha256=4e094c64ab7f22636fc5ef05e64e6529a38d6f3920c7ab1e54d3bb7d4fd31f2e
  Stored in directory: /root/.cache/pip/wheels/5b/d7/fe/ab61f3bf30a350feab3bb4dccd63932d56cbbd32b9ec0d94fa
Successfully built ncpol2sdpa
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 3.3 MB/s eta 0:00:00


# NCPOLR_Step2 ##
* NCPOLR (estimate_1hidden, estimate_2hidden)
* ANM_NCPO (learn, anmNCPO_estimate)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from itertools import combinations
from castle.common import BaseLearner, Tensor
from castle.common.independence_tests import hsic_test
# from inputlds import*
# from functions import*
from ncpol2sdpa import*
from math import sqrt


class NCPOLR(object):
    """Estimator based on NCPOP Regressor

    References
    ----------
    Quan Zhou https://github.com/Quan-Zhou/Proper-Learning-of-LDS/blob/master/ncpop/functions.py

    Examples
    --------
    """

    def __init__(self, **kwargs):
        super(NCPOLR, self).__init__()

    def estimate_1hidden(self, X, Y):
        """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals.
        The module converts a noncommutative optimization problem provided in SymPy
        format to an SDPA semidefinite programming problem.

        Parameters
        ----------
        X : array
            Variable seen as cause
        Y: array
            Variable seen as effect

        Returns
        -------
        y_predict: array
            regression predict values of y or residuals
        """

        T = len(Y)
        level = 1

        # Decision Variables
        G = generate_operators("G", n_vars=1, hermitian=True, commutative=False)[0]
        f = generate_operators("f", n_vars=T, hermitian=True, commutative=False)
        n = generate_operators("m", n_vars=T, hermitian=True, commutative=False)
        p = generate_operators("p", n_vars=T, hermitian=True, commutative=False)

        # Objective
        obj = sum((Y[i]-f[i])**2 for i in range(T)) + 0.5*sum(p[i] for i in range(T))

        # Constraints
        ine1 = [f[i] - G*X[i] - n[i] for i in range(T)]
        ine2 = [-f[i] + G*X[i] + n[i] for i in range(T)]
        ine3 = [p[i]-n[i] for i in range(T)]
        ine4 = [p[i]+n[i] for i in range(T)]
        ines = ine1+ine2+ine3+ine4

        # Solve the NCPO
        sdp = SdpRelaxation(variables = flatten([G,f,n,p]),verbose = 1)
        sdp.get_relaxation(level, objective=obj, inequalities=ines)
        sdp.solve(solver='mosek')
        #sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:/Users/zhouq/Documents/sdpa7-windows/sdpa.exe"})
        print(sdp.primal, sdp.dual, sdp.status)

        if(sdp.status != 'infeasible'):
            print('ok.')
            est_noise = []
            for i in range(T):
                est_noise.append(sdp[n[i]])
            print(est_noise)
            return est_noise
        else:
            print('Cannot find feasible solution.')
            return

    def estimate_2hidden(self, X, Y):
        """Fit Estimator based on NCPOP Regressor model with 2*2 hidden state matrix and predict y or produce residuals.
        The module converts a noncommutative optimization problem provided in SymPy
        format to an SDPA semidefinite programming problem.

        Parameters
        ----------
        X : array
            Variable seen as cause
        Y: array
            Variable seen as effect

        Returns
        -------
        y_predict: array
            regression predict values of y or residuals
        """

        T = len(Y)
        level = 1

        # Decision Variables
        G = generate_operators("G", n_vars=1, hermitian=True, commutative=False)[0]
        H = generate_operators("H", n_vars=1, hermitian=True, commutative=False)[0]
        Fdash = generate_operators("Fdash", n_vars=2, hermitian=True, commutative=False)
        m = generate_operators("m", n_vars=T, hermitian=True, commutative=False)
        q = generate_operators("q", n_vars=T, hermitian=True, commutative=False)
        p = generate_operators("p", n_vars=T, hermitian=True, commutative=True)
        f = generate_operators("f", n_vars=T, hermitian=True, commutative=True)
        n = generate_operators("n", n_vars=T, hermitian=True, commutative=False)
        u = generate_operators("u", n_vars=T, hermitian=True, commutative=False)

        # Objective
        obj = sum((Y[i]-f[i])**2 for i in range(T)) + 0.0005*sum(p[i]**2 for i in range(T)) + 0.001*sum(q[i]**2 for i in range(T))

        # Constraints
        ine1 = [ f[i] - Fdash[0]*m[i]  - Fdash[1]*m[i-1] - p[i] for i in range(1,T)]
        ine2 = [-f[i] + Fdash[0]*m[i] +Fdash[1]*m[i-1] + p[i] for i in range(1,T)]
        ine3 = [ m[i] - G*m[i-1]  - H*m[i-2] - q[i] for i in range(2,T)]
        ine4 = [-m[i] + G*m[i-1] + H*m[i-2] + q[i] for i in range(2,T)]
        ine5 = [p[i] + n[i] for i in range(T)]
        ine6 = [p[i] - n[i] for i in range(T)]
        ine7 = [q[i] + u[i] for i in range(T)]
        ine8 = [q[i] - u[i] for i in range(T)]

        ines = ine1+ine2+ine3+ine4

        # Solve the NCPO
        sdp = SdpRelaxation(variables = flatten([G,H,Fdash,f,p,m,q]),verbose = 1)
        sdp.get_relaxation(level, objective=obj, inequalities=ines)
        sdp.solve(solver='mosek')
        #sdp.solve(solver='sdpa', solverparameters={"executable":"sdpa_gmp","executable": "C:/Users/zhouq/Documents/sdpa7-windows/sdpa.exe"})
        print(sdp.primal, sdp.dual, sdp.status)

        if(sdp.status != 'infeasible'):
            print('ok.')
            est_noise = []
            for i in range(T):
                est_noise.append(sdp[Y[i]-f[i]+q[i]])
            print(est_noise)
            return est_noise
        else:
            print('Cannot find feasible solution.')
            return



class ANM_NCPO(BaseLearner):
    """
    Nonlinear causal discovery with additive noise models

    Use Estimator based on NCPOP Regressor and independent Gaussian noise,
    For the independence test, we implemented the HSIC with a Gaussian kernel,
    where we used the gamma distribution as an approximation for the
    distribution of the HSIC under the null hypothesis of independence
    in order to calculate the p-value of the test result.

    References
    ----------
    Hoyer, Patrik O and Janzing, Dominik and Mooij, Joris M and Peters,
    Jonas and Schölkopf, Bernhard,
    "Nonlinear causal discovery with additive noise models", NIPS 2009

    Parameters
    ----------
    alpha : float, default 0.05
        significance level be used to compute threshold

    Attributes
    ----------
    causal_matrix : array like shape of (n_features, n_features)
        Learned causal structure matrix.

    Examples
    --------
    """

    def __init__(self, alpha=0.05):
        super(ANM_NCPO, self).__init__()
        self.alpha = alpha

    def learn(self, data,causalmodelling, columns=None, regressor=NCPOLR(),test_method=hsic_test, **kwargs):
        """Set up and run the ANM_NCPOP algorithm.

        Parameters
        ----------
        data: numpy.ndarray or Tensor
            Training data.
        causalmodelling: Modelling type(hidden_state1,hidden_state2,ARMA,Discrete)
        columns : Index or array-like
            Column labels to use for resulting tensor. Will default to
            RangeIndex (0, 1, 2, ..., n) if no column labels are provided.
        regressor: Class
            Nonlinear regression estimator, if not provided, it is NCPOLR.
            If user defined, must implement `estimate` self.method. such as :
                `regressor.estimate(x, y)`
        test_method: callable, default test_method
            independence test self.method, if not provided, it is HSIC.
            If user defined, must accept three arguments--x, y and keyword
            argument--alpha. such as :
                `test_method(x, y, alpha=0.05)`
        """

        self.regressor = regressor

        # create learning model and ground truth model
        data = Tensor(data, columns=columns)

        node_num = data.shape[1]
        self.causal_matrix = Tensor(np.zeros((node_num, node_num)),
                                    index=data.columns,
                                    columns=data.columns)

        for i, j in combinations(range(node_num), 2):
            x = data[:, i]
            y = data[:, j]
            xx = x.reshape((-1, 1))
            yy = y.reshape((-1, 1))

            flag = test_method(xx, yy, alpha=self.alpha)
            if flag == 1:
                continue
            # test x-->y
            flag = self.anmNCPO_estimate(x, y, causalmodelling, regressor = regressor, test_method=test_method)
            if flag:
                self.causal_matrix[i, j] = 1
            # test y-->x
            flag = self.anmNCPO_estimate(y, x, causalmodelling, regressor = regressor, test_method=test_method)
            if flag:
                self.causal_matrix[j, i] = 1

    def anmNCPO_estimate(self, x, y, causalmodelling, regressor=NCPOLR(), test_method=hsic_test):
        """Compute the fitness score of the ANM_NCPOP Regression model in the x->y direction.

        Parameters
        ----------
        x: array
            Variable seen as cause
        y: array
            Variable seen as effect
        causalmodelling: Modelling type(hidden_state1,hidden_state2,ARMA,Discrete)
        regressor: Class
            Nonlinear regression estimator, if not provided, it is NCPOP.
            If user defined, must implement `estimate` self.method. such as :
                `regressor.estimate(x, y)`
        test_method: callable, default test_method
            independence test self.method, if not provided, it is HSIC.
            If user defined, must accept three arguments--x, y and keyword
            argument--alpha. such as :
                `test_method(x, y, alpha=0.05)`
        Returns
        -------
        out: int, 0 or 1
            If 1, residuals n is independent of x, then accept x --> y
            If 0, residuals n is not independent of x, then reject x --> y

        """

        x = scale(x)
        y = scale(y)
        if causalmodelling == 'hidden_state1':
            y_res = regressor.estimate_1hidden(x, y)
        elif causalmodelling == 'hidden_state2':
            y_res = regressor.estimate_2hidden(x, y)
        else:
            print('WRONG MODELLING TYPE')
        #if causalmodelling='ARMA':
        #if causalmodelling='Discrete':


        flag = test_method(np.asarray(y_res).reshape((-1, 1)), np.asarray(x).reshape((-1, 1)), alpha=self.alpha)
        print(flag)

        return flag





# Test_BuiltinDataSet_Step3
* erdos_renyi function
Set Gauss_linear/Nonlinear, nodes, edges and DataSize

In [ ]:
# coding=utf-8
# 2020.12 added (1) low rank DAG generations;
#               (2) quad functons for causal functions;
#               (3) event-type data
# 2021.08 deleted (1) condition: sem_type == 'poisson'
# Huawei Technologies Co., Ltd.
#
# Copyright (C) 2021. Huawei Technologies Co., Ltd. All rights reserved.
#
# Copyright (c) Xun Zheng (https://github.com/xunzheng/notears)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
import random
from random import sample
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
from tqdm import tqdm
from copy import deepcopy
from itertools import combinations
from scipy.special import expit as sigmoid


def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)


class DAG(object):
    '''
    A class for simulating random (causal) DAG, where any DAG generator
    method would return the weighed/binary adjacency matrix of a DAG.
    Besides, we recommend using the python package "NetworkX"
    to create more structures types.
    '''

    @staticmethod
    def _random_permutation(M):
        # np.random.permutation permutes first axis only
        P = np.random.permutation(np.eye(M.shape[0]))
        return P.T @ M @ P

    @staticmethod
    def _random_acyclic_orientation(B_und):
        B = np.tril(DAG._random_permutation(B_und), k=-1)
        B_perm = DAG._random_permutation(B)
        return B_perm

    @staticmethod
    def _graph_to_adjmat(G):
        return nx.to_numpy_array(G)

    @staticmethod
    def _BtoW(B, d, w_range):
        U = np.random.uniform(low=w_range[0], high=w_range[1], size=[d, d])
        U[np.random.rand(d, d) < 0.5] *= -1
        W = (B != 0).astype(float) * U
        return W

    @staticmethod
    def _low_rank_dag(d, degree, rank):
        """
        Simulate random low rank DAG with some expected degree.

        Parameters
        ----------
        d: int
            Number of nodes.
        degree: int
            Expected node degree, in + out.
        rank: int
            Maximum rank (rank < d-1).

        Return
        ------
        B: np.nparray
            Initialize DAG.
        """
        prob = float(degree) / (d - 1)
        B = np.triu((np.random.rand(d, d) < prob).astype(float), k=1)
        total_edge_num = np.sum(B == 1)
        sampled_pa = sample(range(d - 1), rank)
        sampled_pa.sort(reverse=True)
        sampled_ch = []
        for i in sampled_pa:
            candidate = set(range(i + 1, d))
            candidate = candidate - set(sampled_ch)
            sampled_ch.append(sample(candidate, 1)[0])
            B[i, sampled_ch[-1]] = 1
        remaining_pa = list(set(range(d)) - set(sampled_pa))
        remaining_ch = list(set(range(d)) - set(sampled_ch))
        B[np.ix_(remaining_pa, remaining_ch)] = 0
        after_matching_edge_num = np.sum(B == 1)

        # delta = total_edge_num - after_matching_edge_num
        # mask B
        maskedB = B + np.tril(np.ones((d, d)))
        maskedB[np.ix_(remaining_pa, remaining_ch)] = 1
        B[maskedB == 0] = 1

        remaining_ch_set = set([i + d for i in remaining_ch])
        sampled_ch_set = set([i + d for i in sampled_ch])
        remaining_pa_set = set(remaining_pa)
        sampled_pa_set = set(sampled_pa)

        edges = np.transpose(np.nonzero(B))
        edges[:, 1] += d
        bigraph = nx.Graph()
        bigraph.add_nodes_from(range(2 * d))
        bigraph.add_edges_from(edges)
        M = nx.bipartite.maximum_matching(bigraph, top_nodes=range(d))
        while len(M) > 2 * rank:
            keys = set(M.keys())
            rmv_cand = keys & (remaining_pa_set | remaining_ch_set)
            p = sample(rmv_cand, 1)[0]
            c = M[p]
            # destroy p-c
            bigraph.remove_edge(p, c)
            M = nx.bipartite.maximum_matching(bigraph, top_nodes=range(d))

        new_edges = np.array(bigraph.edges)
        for i in range(len(new_edges)):
            new_edges[i,].sort()
        new_edges[:, 1] -= d

        BB = np.zeros((d, d))
        B = np.zeros((d, d))
        BB[new_edges[:, 0], new_edges[:, 1]] = 1

        if np.sum(BB == 1) > total_edge_num:
            delta = total_edge_num - rank
            BB[sampled_pa, sampled_ch] = 0
            rmv_cand_edges = np.transpose(np.nonzero(BB))
            if delta <= 0:
                raise RuntimeError(r'Number of edges is below the rank, please \
                                   set a larger edge or degree \
                                   (you can change seed or increase degree).')
            selected = np.array(sample(rmv_cand_edges.tolist(), delta))
            B[selected[:, 0], selected[:, 1]] = 1
            B[sampled_pa, sampled_ch] = 1
        else:
            B = deepcopy(BB)

        B = B.transpose()
        return B

    @staticmethod
    def erdos_renyi(n_nodes, n_edges, weight_range=None, seed=None):

        assert n_nodes > 0
        set_random_seed(seed)
        # Erdos-Renyi
        creation_prob = (2 * n_edges) / (n_nodes ** 2)
        G_und = nx.erdos_renyi_graph(n=n_nodes, p=creation_prob, seed=seed)
        B_und = DAG._graph_to_adjmat(G_und)
        B = DAG._random_acyclic_orientation(B_und)
        if weight_range is None:
            return B
        else:
            W = DAG._BtoW(B, n_nodes, weight_range)
        return W

    @staticmethod
    def scale_free(n_nodes, n_edges, weight_range=None, seed=None):

        assert (n_nodes > 0 and n_edges >= n_nodes and n_edges < n_nodes * n_nodes)
        set_random_seed(seed)
        # Scale-free, Barabasi-Albert
        m = int(round(n_edges / n_nodes))
        G_und = nx.barabasi_albert_graph(n=n_nodes, m=m)
        B_und = DAG._graph_to_adjmat(G_und)
        B = DAG._random_acyclic_orientation(B_und)
        if weight_range is None:
            return B
        else:
            W = DAG._BtoW(B, n_nodes, weight_range)
        return W

    @staticmethod
    def bipartite(n_nodes, n_edges, split_ratio = 0.2, weight_range=None, seed=None):

        assert n_nodes > 0
        set_random_seed(seed)
        # Bipartite, Sec 4.1 of (Gu, Fu, Zhou, 2018)
        n_top = int(split_ratio * n_nodes)
        n_bottom = n_nodes -  n_top
        creation_prob = n_edges/(n_top*n_bottom)
        G_und = bipartite.random_graph(n_top, n_bottom, p=creation_prob, directed=True)
        B_und = DAG._graph_to_adjmat(G_und)
        B = DAG._random_acyclic_orientation(B_und)
        if weight_range is None:
            return B
        else:
            W = DAG._BtoW(B, n_nodes, weight_range)
        return W

    @staticmethod
    def hierarchical(n_nodes, degree=5, graph_level=5, weight_range=None, seed=None):

        assert n_nodes > 1
        set_random_seed(seed)
        prob = float(degree) / (n_nodes - 1)
        B = np.tril((np.random.rand(n_nodes, n_nodes) < prob).astype(float), k=-1)
        point = sample(range(n_nodes - 1), graph_level - 1)
        point.sort()
        point = [0] + [x + 1 for x in point] + [n_nodes]
        for i in range(graph_level):
            B[point[i]:point[i + 1], point[i]:point[i + 1]] = 0
        if weight_range is None:
            return B
        else:
            W = DAG._BtoW(B, n_nodes, weight_range)
        return W

    @staticmethod
    def low_rank(n_nodes, degree=1, rank=5, weight_range=None, seed=None):

        assert n_nodes > 0
        set_random_seed(seed)
        B = DAG._low_rank_dag(n_nodes, degree, rank)
        if weight_range is None:
            return B
        else:
            W = DAG._BtoW(B, n_nodes, weight_range)
        return W


class IIDSimulation(object):
    '''
    Simulate IID datasets for causal structure learning.

    Parameters
    ----------
    W: np.ndarray
        Weighted adjacency matrix for the target causal graph.
    n: int
        Number of samples for standard trainning dataset.
    method: str, (linear or nonlinear), default='linear'
        Distribution for standard trainning dataset.
    sem_type: str
        gauss, exp, gumbel, uniform, logistic (linear);
        mlp, mim, gp, gp-add, quadratic (nonlinear).
    noise_scale: float
        Scale parameter of noise distribution in linear SEM.
    '''

    def __init__(self, W, n=1000, method='linear',
                 sem_type='gauss', noise_scale=1.0):

        self.B = (W != 0).astype(int)
        if method == 'linear':
            self.X = IIDSimulation._simulate_linear_sem(
                    W, n, sem_type, noise_scale)
        elif method == 'nonlinear':
            self.X = IIDSimulation._simulate_nonlinear_sem(
                    W, n, sem_type, noise_scale)
        logging.info('Finished synthetic dataset')

    @staticmethod
    def _simulate_linear_sem(W, n, sem_type, noise_scale):
        """
        Simulate samples from linear SEM with specified type of noise.
        For uniform, noise z ~ uniform(-a, a), where a = noise_scale.

        Parameters
        ----------
        W: np.ndarray
            [d, d] weighted adj matrix of DAG.
        n: int
            Number of samples, n=inf mimics population risk.
        sem_type: str
            gauss, exp, gumbel, uniform, logistic.
        noise_scale: float
            Scale parameter of noise distribution in linear SEM.

        Return
        ------
        X: np.ndarray
            [n, d] sample matrix, [d, d] if n=inf
        """
        def _simulate_single_equation(X, w, scale):
            """X: [n, num of parents], w: [num of parents], x: [n]"""
            if sem_type == 'gauss':
                z = np.random.normal(scale=scale, size=n)
                x = X @ w + z
            elif sem_type == 'exp':
                z = np.random.exponential(scale=scale, size=n)
                x = X @ w + z
            elif sem_type == 'gumbel':
                z = np.random.gumbel(scale=scale, size=n)
                x = X @ w + z
            elif sem_type == 'uniform':
                z = np.random.uniform(low=-scale, high=scale, size=n)
                x = X @ w + z
            elif sem_type == 'logistic':
                x = np.random.binomial(1, sigmoid(X @ w)) * 1.0
            else:
                raise ValueError('Unknown sem type. In a linear model, \
                                 the options are as follows: gauss, exp, \
                                 gumbel, uniform, logistic.')
            return x

        d = W.shape[0]
        if noise_scale is None:
            scale_vec = np.ones(d)
        elif np.isscalar(noise_scale):
            scale_vec = noise_scale * np.ones(d)
        else:
            if len(noise_scale) != d:
                raise ValueError('noise scale must be a scalar or has length d')
            scale_vec = noise_scale
        G_nx =  nx.from_numpy_array(W, create_using=nx.DiGraph)
        if not nx.is_directed_acyclic_graph(G_nx):
            raise ValueError('W must be a DAG')
        if np.isinf(n):  # population risk for linear gauss SEM
            if sem_type == 'gauss':
                # make 1/d X'X = true cov
                X = np.sqrt(d) * np.diag(scale_vec) @ np.linalg.inv(np.eye(d) - W)
                return X
            else:
                raise ValueError('population risk not available')
        # empirical risk
        ordered_vertices = list(nx.topological_sort(G_nx))
        assert len(ordered_vertices) == d
        X = np.zeros([n, d])
        for j in ordered_vertices:
            parents = list(G_nx.predecessors(j))
            X[:, j] = _simulate_single_equation(X[:, parents], W[parents, j], scale_vec[j])
        return X

    @staticmethod
    def _simulate_nonlinear_sem(W, n, sem_type, noise_scale):
        """
        Simulate samples from nonlinear SEM.

        Parameters
        ----------
        B: np.ndarray
            [d, d] binary adj matrix of DAG.
        n: int
            Number of samples.
        sem_type: str
            mlp, mim, gp, gp-add, or quadratic.
        noise_scale: float
            Scale parameter of noise distribution in linear SEM.

        Return
        ------
        X: np.ndarray
            [n, d] sample matrix
        """
        if sem_type == 'quadratic':
            return IIDSimulation._simulate_quad_sem(W, n, noise_scale)

        def _simulate_single_equation(X, scale):
            """X: [n, num of parents], x: [n]"""
            z = np.random.normal(scale=scale, size=n)
            pa_size = X.shape[1]
            if pa_size == 0:
                return z
            if sem_type == 'mlp':
                hidden = 100
                W1 = np.random.uniform(low=0.5, high=2.0, size=[pa_size, hidden])
                W1[np.random.rand(*W1.shape) < 0.5] *= -1
                W2 = np.random.uniform(low=0.5, high=2.0, size=hidden)
                W2[np.random.rand(hidden) < 0.5] *= -1
                x = sigmoid(X @ W1) @ W2 + z
            elif sem_type == 'mim':
                w1 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
                w1[np.random.rand(pa_size) < 0.5] *= -1
                w2 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
                w2[np.random.rand(pa_size) < 0.5] *= -1
                w3 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
                w3[np.random.rand(pa_size) < 0.5] *= -1
                x = np.tanh(X @ w1) + np.cos(X @ w2) + np.sin(X @ w3) + z
            elif sem_type == 'gp':
                from sklearn.gaussian_process import GaussianProcessRegressor
                gp = GaussianProcessRegressor()
                x = gp.sample_y(X, random_state=None).flatten() + z
            elif sem_type == 'gp-add':
                from sklearn.gaussian_process import GaussianProcessRegressor
                gp = GaussianProcessRegressor()
                x = sum([gp.sample_y(X[:, i, None], random_state=None).flatten()
                        for i in range(X.shape[1])]) + z
            else:
                raise ValueError('Unknown sem type. In a nonlinear model, \
                                 the options are as follows: mlp, mim, \
                                 gp, gp-add, or quadratic.')
            return x

        B = (W != 0).astype(int)
        d = B.shape[0]
        if noise_scale is None:
            scale_vec = np.ones(d)
        elif np.isscalar(noise_scale):
            scale_vec = noise_scale * np.ones(d)
        else:
            if len(noise_scale) != d:
                raise ValueError('noise scale must be a scalar or has length d')
            scale_vec = noise_scale

        X = np.zeros([n, d])
        G_nx =  nx.from_numpy_array(B, create_using=nx.DiGraph)
        ordered_vertices = list(nx.topological_sort(G_nx))
        assert len(ordered_vertices) == d
        for j in ordered_vertices:
            parents = list(G_nx.predecessors(j))
            X[:, j] = _simulate_single_equation(X[:, parents], scale_vec[j])
        return X

    @staticmethod
    def _simulate_quad_sem(W, n, noise_scale):
        """
        Simulate samples from SEM with specified type of noise.
        Coefficient is randomly drawn but specifically designed
        to avoid overflow issues.

        Parameters
        ----------
        W: np.ndarray
            weigthed DAG.
        n: int
            Number of samples.
        noise_scale: float
            Scale parameter of noise distribution in linear SEM.

        Return
        ------
        X: np.ndarray
            [n,d] sample matrix
        """
        def generate_quadratic_coef(random_zero=True):
            if random_zero and np.random.randint(low=0, high=2):
                return 0
            else:
                coef = np.random.uniform(low=0.5, high=1)
                if np.random.randint(low=0, high=2):
                    coef *= -1
                return coef

        G = nx.DiGraph(W)
        d = W.shape[0]
        X = np.zeros([n, d])
        ordered_vertices = list(nx.topological_sort(G))
        assert len(ordered_vertices) == d
        for j in ordered_vertices:
            parents = list(G.predecessors(j))

            if len(parents) == 0:
                eta = np.zeros([n])
            elif len(parents) == 1:
                # We don't generate random zero coefficient if there is only one parent
                eta = np.zeros([n])
                used_parents = set()
                p = parents[0]
                num_terms = 0

                # Linear term
                coef = generate_quadratic_coef(random_zero=False)
                if coef != 0:
                    eta += coef * X[:, p]
                    used_parents.add(p)
                    num_terms += 1

                # Squared term
                coef = generate_quadratic_coef(random_zero=False)
                if coef != 0:
                    eta += coef * np.square(X[:, p])
                    used_parents.add(p)
                    num_terms += 1

                if num_terms > 0:
                    eta /= num_terms    # Compute average

                # Remove parent if both coef is zero
                if p not in used_parents:
                    W[p, j] = 0
            else:    # More than 1 parent
                eta = np.zeros([n])
                used_parents = set()
                num_terms = 0

                for p in parents:
                    # Linear terms
                    coef = generate_quadratic_coef(random_zero=True)
                    if coef > 0:
                        eta += coef * X[:, p]
                        used_parents.add(p)
                        num_terms += 1

                    # Squared terms
                    coef = generate_quadratic_coef(random_zero=True)
                    if coef > 0:
                        eta += coef * np.square(X[:, p])
                        used_parents.add(p)
                        num_terms += 1

                # Cross terms
                for p1, p2 in combinations(parents, 2):
                    coef = generate_quadratic_coef(random_zero=True)
                    if coef > 0:
                        eta += coef * X[:, p1] * X[:, p2]
                        used_parents.add(p1)
                        used_parents.add(p2)
                        num_terms += 1

                if num_terms > 0:
                    eta /= num_terms    # Compute average

                # Remove parent if both coef is zero
                unused_parents = set(parents) - used_parents
                if p in unused_parents:
                    W[p, j] = 0

            X[:, j] = eta + np.random.normal(scale=noise_scale, size=n)

        return X


class Topology(object):
    """
    A class for generating some classical (undirected) network structures,
    in which any graph generator method would return the adjacency matrix of
    a network structure.
    In fact, we recommend to directly use the python package "NetworkX"
    to create various structures you need.
    """

    @staticmethod
    def erdos_renyi(n_nodes, n_edges, seed=None):
        """
        Generate topology matrix

        Parameters
        ----------
        n_nodes : int, greater than 0
            The number of nodes.
        n_edges : int, greater than 0
            Use to calculate probability for edge creation.
        seed : integer, random_state, or None (default)
            Indicator of random number generation state.

        Returns
        -------
        B: np.matrix
        """
        assert n_nodes > 0, 'The number of nodes must be greater than 0.'
        creation_prob = (2*n_edges)/(n_nodes**2)
        G = nx.erdos_renyi_graph(n=n_nodes, p=creation_prob, seed=seed)
        B = nx.to_numpy_array(G)
        return B


class THPSimulation(object):
    """
    A class for simulating event sequences with
    THP (Topological Hawkes Process) setting.

    Parameters
    ----------
    causal_matrix: np.matrix
        The casual matrix.
    topology_matrix: np.matrix
        Interpreted as an adjacency matrix to generate graph.
        Has two dimension, should be square.
    mu_range: tuple, default=(0.00005, 0.0001)
    alpha_range: tuple, default=(0.005, 0.007)
    """

    def __init__(self, causal_matrix, topology_matrix,
                 mu_range=(0.00005, 0.0001), alpha_range=(0.005, 0.007)):

        assert (isinstance(causal_matrix, np.ndarray) and
                causal_matrix.ndim == 2 and
                causal_matrix.shape[0] == causal_matrix.shape[1]),\
            'casual_matrix should be np.matrix object, two dimension, square.'
        assert (isinstance(topology_matrix, np.ndarray) and
                topology_matrix.ndim == 2 and
                topology_matrix.shape[0] == topology_matrix.shape[1]),\
            'topology_matrix should be np.matrix object, two dimension, square.'

        self._causal_matrix = (causal_matrix != 0).astype(int)

        self._topo = nx.from_numpy_array(topology_matrix,
                                          create_using=nx.Graph)

        self._mu_range = mu_range
        self._alpha_range = alpha_range

    def simulate(self, T, max_hop=1, beta=10):
        """
        Generate simulation data.
        """
        N = self._causal_matrix.shape[0]

        mu = np.random.uniform(*self._mu_range, N)

        alpha = np.random.uniform(*self._alpha_range, [N, N])
        alpha = alpha * self._causal_matrix
        alpha = np.ones([max_hop+1, N, N]) * alpha

        immigrant_events = dict()
        for node in self._topo.nodes:
            immigrant_events[node] = self._trigger_events(mu, 0, T, beta)

        base_events = immigrant_events.copy()
        events = immigrant_events.copy()
        while sum(map(len, base_events.values())) != 0:
            offspring_events = dict()
            for node in tqdm(self._topo.nodes):
                offspring_events[node] = []
                for k in range(max_hop+1):
                    k_base_events = []
                    for neighbor in self._get_k_hop_neighbors(
                            self._topo, node, k):
                        k_base_events += base_events[neighbor]
                    k_new_events = [self._trigger_events(
                        alpha[k, i], start_time, duration, beta)
                        for (i, start_time, duration) in k_base_events]
                    for event_group in k_new_events:
                        offspring_events[node] += event_group
                events[node] += offspring_events[node]
            base_events = offspring_events

        Xn_list = []
        for node, event_group in events.items():
            Xn = pd.DataFrame(event_group,
                              columns=['event', 'timestamp', 'duration'])
            Xn.insert(0, 'node', node)
            Xn_list.append(Xn.reindex(columns=['event', 'timestamp', 'node']))
        X = pd.concat(Xn_list, sort=False, ignore_index=True)
        return X

    @staticmethod
    def _trigger_events(intensity_vec, start_time, duration, beta):

        events = []
        for i, intensity in enumerate(intensity_vec):
            if intensity:
                trigger_time = start_time
                while True:
                    trigger_time = round(trigger_time + np.random.exponential(
                        1 / intensity))
                    if trigger_time > start_time + duration:
                        break
                    sub_duration = (np.max((0, np.random.exponential(beta)))).round()
                    events.append((i, trigger_time, sub_duration))
        return events

    @staticmethod
    def _get_k_hop_neighbors(G, node, k):
        if k == 0:
            return {node}
        else:
            return (set(nx.single_source_dijkstra_path_length(G, node, k).keys())
                    - set(nx.single_source_dijkstra_path_length(
                        G, node, k - 1).keys()))

# Results and plots_Step4
* Test datasets


# ANM_NCPOP_All


In [ ]:
import os
import re
import csv
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from castle.common import GraphDAG
from castle.metrics import MetricsDAG


class ANM_NCPOP(object):
    '''
    A class for simulating random (causal) DAG from real or synthetic datasets.

    Parameters
    ----------
    data_type: str, (Real or Synthetic), default='Real'
    file_name: str, name of data,
        Setting file_name = 0 if data_type = 'Synthetic'
    method: str, (linear or nonlinear), default='linear'
        Distribution for standard trainning dataset.
    File_PATH: str
        Save file path
    start: int
        Start number of samples for standard trainning dataset
    stop: int
        stop number of samples for standard trainning dataset
    step: int
        step number of samples for standard trainning dataset

    '''

    def __init__(self, data_type, file_name, File_PATH, start, stop, step):
        self.data_type = data_type
        self.file_name = file_name
        self.File_PATH = File_PATH
        self.start = start
        self.stop = stop
        self.step = step

    @staticmethod
    def Ancpop_Test(self):
        ##################################################### Read Dataset ##########################################
        if self.data_type=='Real':
            if not os.path.exists(self.File_PATH):
                print('Dataset does not exist!')
            td_results=os.listdir(self.File_PATH)
            if len(td_results)==0:
                print('INFO: Empty data file!')
            else:
                print('INFO: Reading Data in File!')
                for file_name in td_results:
                    rt = Ancpop_Real(self.File_PATH, self.file_name, self.start, self.stop, self.step)
                    rt.Ancpop_real_Test()

        else:
            for method in ['linear','nonlinear']:
                if method == 'linear':
                    st = ['gauss', 'exp', 'gumbel', 'uniform', 'logistic']
                else:
                    st = ['mlp', 'mim','gp', 'gp-add', 'quadratic']
                for sem_type in st:
                    nodes = range(6,14,3)
                    edges = range(10,21,5)
                    st = Ancpop_Simulation(method, self.File_PATH, sem_type, nodes, edges, self.start, self.stop, self.step)
                    st.Ancpop_simulation_Test()

class Ancpop_Real(object):
    '''
    A class for simulating random (causal) DAG, where any DAG generator
    self.method would return the weighed/binary adjacency matrix of a DAG.
    Besides, we recommend using the python package "NetworkX"
    to create more structures types.

    Parameters
    ----------
    File_PATH: str
        Save file path
    File_NAME: str
        Read data name
    start: int
        Start number of samples for standard trainning dataset
    stop: int
        stop number of samples for standard trainning dataset
    step: int
        step number of samples for standard trainning dataset

    '''
    def __init__(self, File_PATH,File_NAME, start, stop, step):
        self.File_PATH = File_PATH
        self.File_NAME = File_NAME
        self.start = start
        self.stop = stop
        self.step = step
        self.datasize = range(self.start, self.stop, self.step)
        self.datasize_num = len(self.datasize)
        self.sname = re.split("\.", self.File_NAME)[0]


    def Ancpop_real_Test(self):
        ################################################  Test Data #############################################
        self.File_PATH_Heatmaps = self.File_PATH + 'Results_'+self.sname+ '/'
        self.Table_PATH_Summary = self.File_PATH_Heatmaps + 'Summary_'+self.sname+'.csv'
        if not os.path.exists(self.Table_PATH_Summary):
            print('INFO: Testing '+ self.sname+'!')
            self.File_PATH_Base = self.File_PATH + 'Details_'+self.sname+ '/'
            self.Ancpop_real_estimate(self)
        else:
            print('INFO: Finished '+ self.sname+'Sampling!')

        ################################################  Create And Plot #############################################
        #self.Plots_ANCPOP_Real(self)
        print('INFO: Finished plotting '+ self.sname + '!')

    @staticmethod
    def Ancpop_real_estimate(self):
        read_Dir=os.listdir(self.File_PATH)
        if len(read_Dir) == 1:
            self.File_TYPE = re.split("\.", self.File_NAME)[1]
            if self.File_TYPE =='npz':
                print(self.File_TYPE)
                Tests_data = np.load(self.File_PATH+self.File_NAME, allow_pickle=True)
                Raw_data = Tests_data['x']
                true_dag = Tests_data['y']
                #print(Tests_data['x'][:20], Raw_data[:20],true_dag)
                #self.nodes_num = len(causal_matrix)
                #self.edges_num = len(causal_matrix[causal_matrix == 1])
            elif self.File_TYPE =='tar':
                Tests_data = np.load(self.File_NAME+'.tar', allow_pickle=True)
            else:
                print('INFO: Cannot Read '+self.File_TYPE+' Data!')
        else:
            Raw_data_path = self.File_PATH+read_Dir[0]
            Raw_data = pd.read_csv(Raw_data_path, header=0, index_col=0)
            DAG_data_path = self.File_PATH+read_Dir[1]
            DAG_data = pd.read_csv(DAG_data_path, header=0, index_col=0)


        ############################################## Create Files ###################################
        if not os.path.exists(self.File_PATH_Base):
            os.makedirs(self.File_PATH_Base)
        self.File_PATH_MetricsDAG = self.File_PATH_Base +'MetricsDAG/'
        if not os.path.exists(self.File_PATH_MetricsDAG):
            os.makedirs(self.File_PATH_MetricsDAG)
        if not os.path.exists(self.File_PATH_Heatmaps):
            os.makedirs(self.File_PATH_Heatmaps)

        duration_anm_ncpop = []
        f1_anm_ncpop = []
        df = pd.DataFrame(columns=['fdr', 'tpr', 'fpr', 'shd', 'nnz', 'precision', 'recall', 'F1', 'gscore'])

        #df = pd.DataFrame(columns=['DataSize','False_Discovery_Rate', 'True_Positive_Rate', 'False_Positive_Rate', 'SHD', 'NNZ', 'Precision', 'Recall', 'F1_Score', 'G_score'])
        for i in self.datasize:
            data = Raw_data[:i]
            t_start = time.time()
            anmNCPO = ANM_NCPO(alpha=0.05)
            anmNCPO.learn(data = data, causalmodelling = 'hidden_state1')
            # plot predict_dag and true_dag
            GraphDAG(anmNCPO.causal_matrix, true_dag, show=False, save_name = self.File_PATH_MetricsDAG+self.sname+'_' + str(i) + 'Datasize.png')
            met = MetricsDAG(anmNCPO.causal_matrix, true_dag)
            #f1_result.to_csv(self.File_PATH_Heatmaps + 'F1_'+self.sname+'.csv',index=False)
            if math.isnan(float(met.metrics['F1'])):
              f1_anm_ncpop.append(0.2)
            else:
              print(math.isnan(met.metrics['F1']),met.metrics['F1']==np.nan)
              f1_anm_ncpop.append(met.metrics['F1'])
            duration_anm_ncpop.append(time.time()-t_start)
            print(self.sname+'_' + str(i) + 'Datasize is done!'+'F1 Score is'+ str(met.metrics['F1'])+'.')
            #'Time Duration is'+ str(time.time()-t_start))
            df = pd.concat([df, pd.DataFrame([met.metrics])])
        df.to_csv(self.Table_PATH_Summary, index=False)
        #df = pd.concat({"DataSize":[self.datasize]},{df.loc[:, ['F1']]})
        df_F1 = pd.DataFrame({"DataSize":self.datasize,'F1_Score':f1_anm_ncpop,'Duration':duration_anm_ncpop})
        #f1_result = df.loc[:, ['DataSize','F1_Score']]
        df_F1.to_csv(self.File_PATH_Heatmaps + 'F1_'+self.sname+'.csv',index=False)
        return df_F1

    '''
    @staticmethod
    def Summary_Results(self):
        f1_anm_ncpop = pd.DataFrame()
        tqdm=os.listdir(self.File_PATH_Details)
        for i in range(0,len(tqdm)):
            File_PATH = os.path.join(self.File_PATH_Details,tqdm[i])
            #ds = re.split("D",re.split("edges_",re.split("nodes_", re.split("\.", tqdm[i])[0])[1])[1])[0]
            df = pd.read_csv(File_PATH)
            f1_anm_ncpop_nan = df.loc[:,'F1']
            if len([f1_anm_ncpop_nan == 0]) == len(f1_anm_ncpop_nan):
              f1_anm_ncpop_mean = 0.2
            else:
              f1_anm_ncpop_mean = round(np.nanmean(f1_anm_ncpop_nan), 3)
            f1_anm_ncpop = pd.concat((f1_anm_ncpop, pd.DataFrame([ds, f1_anm_ncpop_mean])), axis=1)
        f1_result = pd.DataFrame(np.array(f1_anm_ncpop.T), columns=['Linear','Gauss','Nodes','Edges','DataSize','F1_Score'])
        f1_result.to_csv(self.Table_PATH_Summary,index=False)
        return f1_result'''

    @staticmethod
    def Plots_ANCPOP_Real(self):
        self.pro_rang = np.arange(self.start, self.stop, self.step)
        self.obs_rang = np.arange(self.start, self.stop, self.step)
        fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(20,20))
        z =[]
        zz = pd.read_csv(self.File_PATH_Heatmaps + 'F1_'+self.sname+'.csv', header=[1], index_col=0)
        z = zz.to_numpy()
        z_min=np.min(z)
        z_max=np.max(z)
        c = axes.imshow(z, cmap =plt.cm.bone_r, vmin = z_min, vmax = z_max,
                        interpolation ='nearest', origin ='upper')
        axes.set_xlabel('length of time windows '+r'$T$',fontsize=10)
        #axes.set_ylabel('F1 score ',fontsize=10)
        positions = range(9)
        labels=self.datasize
        axes.yaxis.set_major_locator(ticker.FixedLocator(positions))
        axes.yaxis.set_major_formatter(ticker.FixedFormatter(labels))
        for ytick in axes.get_yticklabels():
            ytick.set_fontsize(4)
        axes.xaxis.set_major_locator(ticker.FixedLocator(positions))
        axes.xaxis.set_major_formatter(ticker.FixedFormatter(labels))
        for xtick in axes.get_xticklabels():
            xtick.set_fontsize(4)

        fig.colorbar(c, ax=axes.ravel().tolist())
        plt.savefig(self.File_PATH_Heatmaps +'Heatmap_ '+self.sname+'.pdf', bbox_inches='tight')


class Ancpop_Simulation(object):
    '''
    A class for simulating random (causal) DAG, where any DAG generator
    self.method would return the weighed/binary adjacency matrix of a DAG.
    Besides, we recommend using the python package "NetworkX"
    to create more structures types.

    Parameters
    ----------
    method: str, (linear or nonlinear), default='linear'
        Distribution for standard trainning dataset.
    File_PATH_Base: str
        Save file path
    sem_type: str
        gauss, exp, gumbel, uniform, logistic (linear);
        mlp, mim, gp, gp-add, quadratic (nonlinear).
    nodes: series
        Notes of samples for standard trainning dataset
    edges: series
        Edges of samples for standard trainning dataset
    start: int
        Start number of samples for standard trainning dataset
    stop: int
        stop number of samples for standard trainning dataset
    step: int
        step number of samples for standard trainning dataset

    '''

    def __init__(self, method, File_PATH, sem_type, nodes, edges, start, stop, step):
        self.method = method
        self.File_PATH = File_PATH
        self.printname = self.method.capitalize()+' SEM Samples with ' + self.sem_type.capitalize() +' Noise'
        self.filename = self.method.capitalize()+'SEM_' + self.sem_type.capitalize() +'Noise'
        self.File_PATH_Base = self.File_PATH +'Result_'+ self.filename +'/'
        self.sem_type = sem_type
        self.nodes = nodes
        self.edges = edges
        self.start = start
        self.stop = stop
        self.step = step
        self.nodes_num = len(self.nodes)
        self.edges_num = len(self.edges)
        self.pro_rang = np.arange(self.start, self.stop, self.step)
        self.obs_rang = np.arange(self.start, self.stop, self.step)
        self.datasize = range(self.start, self.stop, self.step)
        self.datasize_num = len(self.datasize)

    @staticmethod
    def Ancpop_simulation_Test(self):
        ############################################## Create And Download Dataset ###################################
        self.File_PATH_Details = self.File_PATH_Base + 'Results_Details/'
        if not os.path.exists(self.File_PATH_Details):
            os.makedirs(self.File_PATH_Details)
        print('INFO: Created Result_'+self.method.capitalize()+' File!')
        tqdm_csv=os.listdir(self.File_PATH_Details)
        if len(tqdm_csv) != self.nodes_num*self.edges_num* self.datasize_num:
            print('INFO: Simulating '+self.printname +'!')
            self.Ancpop_simulation_estimate(self, sem_type)
        else:
            print('INFO: Finished '+ self.printname+' simulation!')

        ################################################# Summary Dataset ############################################
        ################################## Insert/Download Dataset And Create Summary Table############################
        self.File_PATH_Heatmaps = self.File_PATH + 'Results_'+self.filename+ '/'
        if not os.path.exists(self.File_PATH_Heatmaps):
            os.makedirs(self.File_PATH_Heatmaps )
        self.Table_PATH_Summary = self.File_PATH_Heatmaps + 'Summary_'+self.filename+'.csv'
        if not os.path.exists(self.Table_PATH_Summary):
            print('INFO: Summarizing samples from '+'!')
            self.Summary_Results(self)
            # Table_Summary = self.Summary_Results()

        else:
            print('INFO: Finished '+ self.printname+' summary!')
            # Table_Summary = pd.read_csv(Table_PATH_Summary,header=0,index_col=0)

        ######################################### Create And Combine Dataset Summary #################################
        self.File_PATH_Results = self.File_PATH_Base + 'Results/'
        if not os.path.exists(self.File_PATH_Results):
            os.makedirs(self.File_PATH_Results)
        tqdm_results=os.listdir(self.File_PATH_Results)
        if len(tqdm_results) != (self.nodes_num+self.edges_num)*2:
            print('INFO: Preparing plot!')
            self.Type_Results(self, 'Nodes')
            self.Type_Results(self, 'Edges')

        ################################################  Create And Plot #############################################
        self.Plots_Type_ANCPOP(self,'Nodes')
        print('INFO: Finished plotting '+ self.printname + ' on nodes!')

        self.Plots_Type_ANCPOP(self,'Edges')
        print('INFO: Finished plotting '+ self.printname + ' on edges!')

        self.Plots_ANCPOP(self)
        print('INFO: Finished plotting '+ self.printname + '!')


    @staticmethod
    def Ancpop_simulation_estimate(self, sem_type):
        duration_anm_ncpop = []
        f1_anm_ncpop = []
        df = pd.DataFrame(columns=['fdr', 'tpr', 'fpr', 'shd', 'nnz', 'precision', 'recall', 'F1', 'gscore'])
        for nn in range(6,14,3):
            for ne in range(10,21,5):
                weighted_random_dag = DAG.erdos_renyi(n_nodes=nn, n_edges=ne, seed=1)
                for ds in range(5,40,5):
                    sname = self.method+ '_gauss_'+str(nn)+'nodes_'+str(ne)+'edges_'+str(ds)+'DataSize'
                    path_check = os.path.exists(self.File_PATH_Base + 'Results_Details/' +sname+'.csv')
                    if not path_check:
                        dataset = IIDSimulation(W=weighted_random_dag, n=ds, method=self.method, sem_type=sem_type)
                        true_dag, data = dataset.B, dataset.X

                        t_start = time.time()
                        anmNCPO = ANM_NCPO(alpha=0.05)
                        anmNCPO.learn(data=data,causalmodelling='hidden_state1')

                        # plot predict_dag and true_dag
                        self.File_PATH_MetricsDAG = self.File_PATH_Base +'MetricsDAG/'
                        if not os.path.exists(self.File_PATH_MetricsDAG):
                          os.makedirs(self.File_PATH_MetricsDAG)
                        GraphDAG(anmNCPO.causal_matrix, true_dag, show=False, save_name = self.File_PATH_MetricsDAG +'Result_' +sname)
                        met = MetricsDAG(anmNCPO.causal_matrix, true_dag)
                        duration_anm_ncpop.append(time.time()-t_start)
                        f1_anm_ncpop.append(met.metrics['F1'])
                        print(sname+ 'is done!'+'F1 Score is'+ str(met.metrics['F1'])+'.' 'Time Duration is'+ str(time.time()-t_start))
                        #mm = pd.DataFrame(pd.DataFrame([mm]).append([met.metrics]).dropna(axis = 0, how ='any'))#.drop_duplicates(inplace= True)
                        df = pd.concat([df, pd.DataFrame([met.metrics])])
                        df.to_csv(self.File_PATH_Details +sname+'.csv', index=False)
                df1=df.assign(DataSize = ds)
                df1.to_csv(self.File_PATH_Base + 'summary_' + self.method+ '_gauss_'+str(nn)+'nodes_'+str(ne)+'edges' +'.csv', index=False)
        f1_anm_ncpop = np.array(f1_anm_ncpop)
        np.savetxt(self.File_PATH_Base + 'Summary_F1_' +self.filename+'.csv', f1_anm_ncpop, delimiter=',')
        #f1_anm_ncpop.to_csv(self.File_PATH_Base + 'Summary_F1_' + self.method +'_Gauss.csv', index=False)
        np.savetxt(self.File_PATH_Base + 'Duration_' +self.filename+'.csv', duration_anm_ncpop, delimiter=',')
        #duration_anm_ncpop.to_csv(self.File_PATH_Base + 'Duration_' + self.method + '_Gauss.csv', index=False)
        #f1_if_lds_mean = np.mean(f1_if_lds, axis=1)
        #f1_if_lds_std = np.std(f1_if_lds, axis=1)
        print(df1,f1_anm_ncpop, duration_anm_ncpop)

    @staticmethod
    def Summary_Results(self):
        f1_anm_ncpop = pd.DataFrame()
        tqdm=os.listdir(self.File_PATH_Details)
        for i in range(0,len(tqdm)):
            File_PATH = os.path.join(self.File_PATH_Details,tqdm[i])
            #entries = re.split("_", re.split("\.", tqdm[i])[0])
            self.method_nn = re.split("_",re.split("nodes_", re.split("\.", tqdm[i])[0])[0])
            ne = re.split("edges_",re.split("nodes_", re.split("\.", tqdm[i])[0])[1])[0]
            ds = re.split("D",re.split("edges_",re.split("nodes_", re.split("\.", tqdm[i])[0])[1])[1])[0]
            df = pd.read_csv(File_PATH)
            f1_anm_ncpop_nan = df.loc[:,'F1']
            if len([f1_anm_ncpop_nan == 0]) == len(f1_anm_ncpop_nan):
              f1_anm_ncpop_mean = 0.2
            else:
              f1_anm_ncpop_mean = round(np.nanmean(f1_anm_ncpop_nan), 3)
            f1_anm_ncpop = pd.concat((f1_anm_ncpop, pd.DataFrame(self.method_nn+[ne, ds, f1_anm_ncpop_mean])), axis=1)
        f1_result = pd.DataFrame(np.array(f1_anm_ncpop.T), columns=['Linear','Gauss','Nodes','Edges','DataSize','F1_Score'])
        f1_result.to_csv(self.Table_PATH_Summary,index=False)
        return f1_result

    @staticmethod
    def Type_Results(self, type):
        if type == 'Nodes':
            pivot = 'Edges'
        else:
            pivot = 'Nodes'
        f1_result =pd.read_csv(self.Table_PATH_Summary, header=0, index_col=0)
        group_obj = f1_result.groupby(type)#.agg('mean')
        for i in group_obj:
            print(i[0])
            f1_anm_ncpop_ = i[1].pivot(index=pivot,columns='DataSize',values='F1_Score')
            f1_anm_ncpop_result = f1_anm_ncpop_.reset_index()
            #print(f1_anm_ncpop_result)
            self.Table_PATH_Results = self.File_PATH_Results + 'Summary_'+type+'_'+str(i[0])
            f1_anm_ncpop_result.to_csv(self.Table_PATH_Results+".csv",index=False)
            np.save(self.Table_PATH_Results+".npy", f1_anm_ncpop_result)

    @staticmethod
    def Plots_Type_ANCPOP(self, type):
        self.pro_rang = np.arange(self.start, self.stop, self.step)
        self.obs_rang = np.arange(self.start, self.stop, self.step)
        if type == 'Nodes':
            type_num = self.nodes_num
            size = self.nodes
            labels = self.edges
            yaxes = 'Number of Edges'
        else:
            type_num = self.edges_num
            size = self.edges
            labels = self.nodes
            yaxes = 'Number of Variables'
        z = [[] for i in range(type_num)]
        pro_rang = self.pro_rang
        obs_rang = self.obs_rang
        fig, axes = plt.subplots(nrows=1, ncols=type_num,figsize=(20,4))
        for i in range(type_num):
            read_path = self.File_PATH_Base + 'Results/Summary_'+ type +'_'+str(size[i]) +'.csv'
            zz = pd.read_csv(read_path,header=0,index_col=0)
            z[i] = zz.to_numpy()
        z_min=np.min(z)
        z_max=np.max(z)
        for i in range(type_num):
            c = axes[i].imshow(z[i], cmap =plt.cm.bone_r, vmin = z_min, vmax = z_max, interpolation ='nearest', origin ='upper')
            axes[i].set_xlabel('length of time windows '+r'$T$',fontsize=10)
            axes[i].set_ylabel(yaxes,fontsize=10)
            axes[i].title.set_text('F1 score of '+ type + str(size[i]))
            positions = range(8)
            axes[i].yaxis.set_major_locator(ticker.FixedLocator(positions))
            axes[i].yaxis.set_major_formatter(ticker.FixedFormatter(labels))
            for ytick in axes[i].get_yticklabels():
                ytick.set_fontsize(4)
                #ytick.set_rotation(45)
            xlabels = self.datasize
            axes[i].xaxis.set_major_locator(ticker.FixedLocator(positions))
            axes[i].xaxis.set_major_formatter(ticker.FixedFormatter(xlabels))
            for xtick in axes[i].get_xticklabels():
                xtick.set_fontsize(4)
                #xtick.set_rotation(45)

        fig.colorbar(c, ax=axes.ravel().tolist())
        plt.savefig(self.File_PATH_Heatmaps +'Heatmap_ '+self.filename+'_'+type+'.pdf', bbox_inches='tight')

    @staticmethod
    def Plots_ANCPOP(self):
        self.pro_rang = np.arange(self.start, self.stop, self.step)
        self.obs_rang = np.arange(self.start, self.stop, self.step)
        max_num = np.max([self.nodes_num, self.edges_num])
        z = [[[] for i in range(max_num)]for j in range(2)]
        # z[1][2] [[[],[],[]],[[],[],[]]]
        pro_rang = self.pro_rang
        obs_rang = self.obs_rang
        fig, axes = plt.subplots(nrows=2, ncols=len(z[0]),figsize=(20,8))
        for j in range(2):
            if j == 0:
                type = 'Nodes'
                size = self.nodes
                labels = self.edges
                yaxes = 'Number of Edges'
            else:
                type = 'Edges'
                size = self.edges
                labels = self.nodes
                yaxes = 'Number of Variables'
            for i in range(max_num):
                read_path = self.File_PATH_Base + 'Results/Summary_'+ type +'_'+str(size[i]) +'.csv'
                zz = pd.read_csv(read_path,header=0,index_col=0)
                z[j][i] = zz.to_numpy()
            z_min=np.min(z[j])
            z_max=np.max(z[j])
            for i in range(max_num):
                c = axes[j][i].imshow(z[j][i], cmap =plt.cm.bone_r, vmin = z_min, vmax = z_max, interpolation ='nearest', origin ='upper')
                axes[j][i].set_xlabel('length of time windows '+r'$T$',fontsize=10)
                axes[j][i].set_ylabel(yaxes,fontsize=10)
                axes[j][i].title.set_text('F1 score of '+ type + str(size[i]))
                positions = range(8)
                axes[j][i].yaxis.set_major_locator(ticker.FixedLocator(positions))
                axes[j][i].yaxis.set_major_formatter(ticker.FixedFormatter(labels))
                for ytick in axes[j][i].get_yticklabels():
                    ytick.set_fontsize(4)
                    #ytick.set_rotation(45)
                xlabels = self.datasize
                axes[j][i].xaxis.set_major_locator(ticker.FixedLocator(positions))
                axes[j][i].xaxis.set_major_formatter(ticker.FixedFormatter(xlabels))
                for xtick in axes[j][i].get_xticklabels():
                    xtick.set_fontsize(4)
                    #xtick.set_rotation(45)
        title = 'Performance of '+ self.printname
        fig.colorbar(c, ax=axes.ravel().tolist())
        fig.suptitle(title, fontsize=16)
        #plt.title(title, fontdict=None, loc='center', pad=None)
        plt.savefig(self.File_PATH_Heatmaps +'Heatmap_ '+self.filename+'.pdf', bbox_inches='tight')


if __name__ == "__main__":
    ######################################## SETTING GAUSS_TYPE, self.File_PATH ######################################
    ########################### SETTING GAUSS_TYPE, self.File_PATH_Base, nodes, edges, and datasize ##########################
    #File_PATH = '/content/drive/MyDrive/Colab Notebooks/Causality_NotesTest/Test_Causality_Datasets/Real_data/Telephone/'
    File_PATH = '/content/drive/MyDrive/Colab Notebooks/Causality_NotesTest/Test_Causality_Datasets/Synthetic datasets/'

    start = 5
    stop = 40
    step = 5
    file_name = 'linearGauss_6_15.npz'
    result = ANM_NCPOP('Real', 0 , File_PATH, start, stop, step)
    result.Ancpop_Test()

TypeError: ignored

# ANM_NCPOP_Realdata
* Make sure there is only one table in file

# Test data

In [ ]:
import pandas as pd
File_PATH = '/content/drive/MyDrive/Colab Notebooks/Causality_NotesTest/Test_Causality_Datasets/Real_data/Telephone/'
Raw_data_path = File_PATH+'real_dataset_processed.csv'
Raw_dag = File_PATH+'true_graph.csv'

Raw_data = pd.read_csv(Raw_data_path, header=0, index_col=0)
#22589 rows × 54 columns
Raw_data[:10]

KeyError: ignored